<a href="https://colab.research.google.com/github/Kensuzuki95/EY_Assignment/blob/main/EY_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download Packages

In [ ]:
import os
import pandas as pd
import xlrd
import openpyxl
import glob
import shutil
import re
!pip install pandas pyxlsb

# Investigate Folder Content

In [ ]:
# folder path
dir_path = r'/content/drive/MyDrive/EY Recruitment Documents/invoice'
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

File count: 4406


In [ ]:
tifCounter = len(glob.glob1('/content/drive/MyDrive/EY Recruitment Documents/invoice',"*.xlsx"))
tifCounter

4400

In [ ]:
tifCounter = len(glob.glob1('/content/drive/MyDrive/EY Recruitment Documents/invoice',"*.xlsb"))
tifCounter

6

## Sort Files depending on Extension Types

In [ ]:
def extract_files_by_extension(source_folder, destination_folder, extension):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Iterate over files in the source folder
    for filename in os.listdir(source_folder):
        if filename.endswith(extension):
            # Get the full path of the source file
            source_file = os.path.join(source_folder, filename)

            # Get the full path of the destination file
            destination_file = os.path.join(destination_folder, filename)

            # Copy the file to the destination folder
            shutil.copy2(source_file, destination_file)

### Create a folder with .xlsx extension Invoices

In [ ]:
# Create a folder with only .xlsx files
source_folder = '/content/drive/MyDrive/EY Recruitment Documents/invoice'
destination_folder = '/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsx'
extension = '.xlsx'

extract_files_by_extension(source_folder, destination_folder, extension)

In [ ]:
tifCounter = len(glob.glob1('/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsx',"*.xlsx"))
tifCounter

4400

### Create a folder with .xlsb extention Invoices

In [ ]:
# Create a folder with only .xlsb files
source_folder = '/content/drive/MyDrive/EY Recruitment Documents/invoice'
destination_folder = '/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb'
extension = '.xlsb'

extract_files_by_extension(source_folder, destination_folder, extension)

In [ ]:
tifCounter = len(glob.glob1('/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb',"*.xlsb"))
tifCounter

6

In [ ]:
def convert_xlsb_to_xlsx(input_folder, output_folder):
    # Get a list of all files in the input folder
    files = os.listdir(input_folder)

    # Filter out only the .xlsb files
    xlsb_files = [file for file in files if file.endswith('.xlsb')]

    # Iterate over the .xlsb files and convert them to .xlsx
    for file in xlsb_files:
        # Construct the input file path
        input_file_path = os.path.join(input_folder, file)

        # Construct the output file path by replacing the extension and adding the output folder
        output_file_path = os.path.join(output_folder, os.path.splitext(file)[0] + '.xlsx')

        # Read the .xlsb file using pandas
        df = pd.read_excel(input_file_path, engine='pyxlsb')

        # Write the DataFrame to the .xlsx file
        df.to_excel(output_file_path, index=False)

        print(f"Converted {input_file_path} to {output_file_path}")

In [ ]:
# Provide the input and output folder paths
input_folder = '/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb'
output_folder = '/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb_to_xlsx'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

convert_xlsb_to_xlsx(input_folder, output_folder)

#Extract Data from Files

In [ ]:
def extract_invoice_data(file_path):
    wb = openpyxl.load_workbook(file_path)
    sheet = wb.active

    invoice_data = {}

    # Extract data from invoice template 1
    if sheet['A1'].value == r'請　求　書': #Check whether the invocie format is template 1 (or template 2)
      invoice_data['file_name'] = os.path.basename(file_path)
      invoice_data['store_name'] = sheet['A2'].value
      invoice_data['invoice_number'] = sheet['G2'].value
      invoice_data['invoice_date'] = sheet['G3'].value
      invoice_data['product_1'] = sheet['A15'].value
      invoice_data['quantity_1'] = sheet['D15'].value
      invoice_data['product_2'] = sheet['A16'].value
      invoice_data['quantity_2'] = sheet['D16'].value
      invoice_data['product_3'] = sheet['A17'].value
      invoice_data['quantity_3'] = sheet['D17'].value #In case there are more than 4 products registered due to human error
      invoice_data['product_4'] = sheet['A18'].value
      invoice_data['quantity_4'] = sheet['D18'].value
      invoice_data['product_5'] = sheet['A19'].value #In case there are more than 4 products registered due to human error
      invoice_data['quantity_5'] = sheet['D19'].value

    # Extract data from invoice template 2
    else:
      invoice_data['file_name'] = os.path.basename(file_path)
      invoice_data['store_name'] = {sheet['B4'].value, sheet['B6'].value}
      invoice_data['invoice_number'] = sheet['H2'].value
      invoice_data['invoice_date'] = sheet['H1'].value
      invoice_data['product_1'] = sheet['B17'].value
      invoice_data['quantity_1'] = sheet['F17'].value
      invoice_data['product_2'] = sheet['B18'].value
      invoice_data['quantity_2'] = sheet['F18'].value
      invoice_data['product_3'] = sheet['B19'].value
      invoice_data['quantity_3'] = sheet['F19'].value #In case there are more than 4 products registered due to human error
      invoice_data['product_4'] = sheet['B20'].value
      invoice_data['quantity_4'] = sheet['F20'].value
      invoice_data['product_5'] = sheet['B21'].value #In case there are more than 4 products registered due to human error
      invoice_data['quantity_5'] = sheet['F21'].value

    wb.close()

    return invoice_data

## Extract Data from .xlsx File

In [ ]:
# Folder path containing the Excel files
folder_path = '/content/drive/MyDrive/EY Recruitment Documents/invoice'
output_folder_path = '/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsx_data.xlsx'
invoice_data_list_xlsx = []

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):  # Consider only .xlsx files
        file_path = os.path.join(folder_path, file_name)
        invoice_data_xlsx = extract_invoice_data(file_path)
        invoice_data_list_xlsx.append(invoice_data_xlsx)

In [ ]:
# Save the invoice data to the output folder
invoice_xlsx = pd.DataFrame(invoice_data_list_xlsx)
invoice_xlsx.to_excel(output_folder_path, index=False)

# Show the invocie data in a DataFrame format
#invoice_xlsx.head()
invoice_xlsx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4400 entries, 0 to 4399
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   file_name       4400 non-null   object 
 1   store_name      4400 non-null   object 
 2   invoice_number  4400 non-null   object 
 3   invoice_date    4400 non-null   object 
 4   product_1       4400 non-null   object 
 5   quantity_1      4400 non-null   float64
 6   product_2       4400 non-null   object 
 7   quantity_2      4400 non-null   float64
 8   product_3       970 non-null    object 
 9   quantity_3      970 non-null    float64
 10  product_4       0 non-null      object 
 11  quantity_4      0 non-null      object 
 12  product_5       0 non-null      object 
 13  quantity_5      0 non-null      object 
dtypes: float64(3), object(11)
memory usage: 481.4+ KB


## Extract Data from .xlsb File

In [ ]:
# Folder path containing the Excel files
folder_path = '/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb_to_xlsx'
output_folder_path = '/content/drive/MyDrive/EY Recruitment Documents/invoice_xlsb_data.xlsx'
invoice_data_list_xlsb = []

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):  # Consider only .xlsb files
        file_path = os.path.join(folder_path, file_name)
        invoice_data_xlsb = extract_invoice_data(file_path)
        invoice_data_list_xlsb.append(invoice_data_xlsb)

In [ ]:
# Save the invoice data to the output folder
invoice_xlsb = pd.DataFrame(invoice_data_list_xlsb)
invoice_xlsb.to_excel(output_folder_path, index=False)

# Show the invocie data in a DataFrame format
invoice_xlsb
#invoice_xlsb.info()

,file_name,store_name,invoice_number,invoice_date,product_1,quantity_1,product_2,quantity_2,product_3,quantity_3,product_4,quantity_4,product_5,quantity_5
0,202003000008681.xlsx,"{イトーヨーカ堂, 我孫子南口店 御中}",請求番号: 202003000008681,請求日: 2020-03-22,メンズ・シャンプーM,100,レディース・シャンプーL,100,None,None,None,None,None,None
1,202004000002111.xlsx,"{イトーヨーカ堂, 市原店 御中}",請求番号: 202004000002111,請求日: 2020-04-19,メンズ・シャンプーM,100,レディース・シャンプーL,100,None,None,None,None,None,None
2,202010000002111.xlsx,"{イトーヨーカ堂, 市原店 御中}",請求番号: 202010000002111,請求日: 2020-10-17,メンズ・シャンプーM,100,レディース・シャンプーL,100,None,None,None,None,None,None
3,202001000002111.xlsx,"{イトーヨーカ堂, 市原店 御中}",請求番号: 202001000002111,請求日: 2020-01-08,メンズ・シャンプーM,100,レディース・シャンプーL,100,None,None,None,None,None,None
4,202006000008681.xlsx,"{イトーヨーカ堂, 我孫子南口店 御中}",請求番号: 202006000008681,請求日: 2020-06-07,メンズ・シャンプーM,100,レディース・シャンプーL,100,None,None,None,None,None,None
5,202012000008681.xlsx,"{イトーヨーカ堂, 我孫子南口店 御中}",請求番号: 202012000008681,請求日: 2020-12-15,メンズ・シャンプーM,100,レディース・シャンプーL,100,None,None,None,None,None,None


## Merge two datasets to create the main dataset

In [ ]:
invoice_df = pd.concat([invoice_xlsx, invoice_xlsb], ignore_index=True)
invoice_df.to_excel('/content/drive/MyDrive/EY Recruitment Documents/invoice_data.xlsx', index=False)
invoice_df.head()
invoice_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4406 entries, 0 to 4405
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   file_name       4406 non-null   object 
 1   store_name      4406 non-null   object 
 2   invoice_number  4406 non-null   object 
 3   invoice_date    4406 non-null   object 
 4   product_1       4406 non-null   object 
 5   quantity_1      4406 non-null   float64
 6   product_2       4406 non-null   object 
 7   quantity_2      4406 non-null   float64
 8   product_3       970 non-null    object 
 9   quantity_3      970 non-null    object 
 10  product_4       0 non-null      object 
 11  quantity_4      0 non-null      object 
 12  product_5       0 non-null      object 
 13  quantity_5      0 non-null      object 
dtypes: float64(2), object(12)
memory usage: 482.0+ KB


# Data Cleaning

In [ ]:
# Remove Unnecesarry Columns
invoice_df = pd.read_excel('/content/drive/MyDrive/EY Recruitment Documents/invoice_data.xlsx')
invoice_df.columns
invoice_df.drop(['product_4', 'quantity_4', 'product_5', 'quantity_5'], axis = 'columns', inplace = True)
invoice_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4406 entries, 0 to 4405
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   file_name       4406 non-null   object 
 1   store_name      4406 non-null   object 
 2   invoice_number  4406 non-null   object 
 3   invoice_date    4406 non-null   object 
 4   product_1       4406 non-null   object 
 5   quantity_1      4406 non-null   int64  
 6   product_2       4406 non-null   object 
 7   quantity_2      4406 non-null   int64  
 8   product_3       970 non-null    object 
 9   quantity_3      970 non-null    float64
dtypes: float64(1), int64(2), object(7)
memory usage: 344.3+ KB


In [ ]:
invoice_xlsx.head()

In [ ]:
invoice_xlsb.head()

In [ ]:
def data_cleansing(string):
    # Convert set to string
    if isinstance(string, set):
        string = list(string)[0]

    # Convert value to string
    string = str(string)

    # Remove curly brackets
    string = string.replace('{', '').replace('}', '')

    # Remove specific strings
    string = string.replace('御中', '')
    string = string.replace('請求番号', '')
    string = string.replace('請求日', '')

    # Remove specific characters
    string = string.replace(':', '')
    string = string.replace(',', '')
    string = string.replace('\'', '')


    #Align Data format
    string = string.replace('/', '-')

    # Remove double spacing
    string = ' '.join(string.split())

    # Convert float to integer
    #try:
    #    string = str(int(float(string)))
    #except ValueError:
    #    pass

    return string

In [ ]:
# Apply the function to the DataFrame column
invoice_df['store_name'] = invoice_df['store_name'].apply(data_cleansing)
invoice_df['invoice_number'] = invoice_df['invoice_number'].apply(data_cleansing)
invoice_df['invoice_date'] = invoice_df['invoice_date'].apply(data_cleansing)
invoice_df.head()

,file_name,store_name,invoice_number,invoice_date,product_1,quantity_1,product_2,quantity_2,product_3,quantity_3
0,202009000096221.xlsx,トモズ お花茶屋店,202009000096221,44091,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN
1,202006000103601.xlsx,トモズ 小田急マルシェ本厚木店,202006000103601,43997,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN
2,202212000012341.xlsx,マツモトキヨシ せんげん台東店,202212000012341,2022-12-09,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN
3,202209000096221.xlsx,トモズ お花茶屋店,202209000096221,2022-09-05,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN
4,202006000096221.xlsx,トモズ お花茶屋店,202006000096221,43991,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN


In [ ]:
invoice_df.info()
invoice_df.to_excel('/content/drive/MyDrive/EY Recruitment Documents/invoice_data_cleaned.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4406 entries, 0 to 4405
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   file_name       4406 non-null   object 
 1   store_name      4406 non-null   object 
 2   invoice_number  4406 non-null   object 
 3   invoice_date    4406 non-null   object 
 4   product_1       4406 non-null   object 
 5   quantity_1      4406 non-null   int64  
 6   product_2       4406 non-null   object 
 7   quantity_2      4406 non-null   int64  
 8   product_3       970 non-null    object 
 9   quantity_3      970 non-null    float64
dtypes: float64(1), int64(2), object(7)
memory usage: 344.3+ KB


## Create Addtional Columns using supplemental data

In [ ]:
invoice_df = pd.read_excel('/content/drive/MyDrive/EY Recruitment Documents/invoice_data_cleaned.xlsx')
invoice_df.head()

,file_name,store_name,invoice_number,invoice_date,product_1,quantity_1,product_2,quantity_2,product_3,quantity_3
0,202009000096221.xlsx,トモズ お花茶屋店,202009000096221,44091,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN
1,202006000103601.xlsx,トモズ 小田急マルシェ本厚木店,202006000103601,43997,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN
2,202212000012341.xlsx,マツモトキヨシ せんげん台東店,202212000012341,2022-12-09,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN
3,202209000096221.xlsx,トモズ お花茶屋店,202209000096221,2022-09-05,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN
4,202006000096221.xlsx,トモズ お花茶屋店,202006000096221,43991,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN


### Add Franchise Column & Company Name Column & Store Name Column

In [ ]:
# Strings to check
brand_name = ['マツモトキヨシ', 'トモズ', 'イトーヨーカ堂']

# Specify the column to check
column_to_check = 'store_name'

# Function to find the matched string
def find_brand_name(row, column):
    for string in brand_name:
        if string in row[column]:
            return string
    return None

In [ ]:
# Add a Company Name Column & Store Name Column
invoice_df['company_name'] = invoice_df.apply(lambda row: find_brand_name(row, column_to_check), axis=1)
invoice_df.head(1)

,file_name,store_name,invoice_number,invoice_date,product_1,quantity_1,product_2,quantity_2,product_3,quantity_3,company_name
0,202009000096221.xlsx,トモズ お花茶屋店,202009000096221,44091,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ


In [ ]:
# Check which company name is contained in Company Name column and determine the franchise name
invoice_df['franchise_name'] = ''
invoice_df.loc[invoice_df['company_name'].str.contains('マツモトキヨシ'), 'franchise_name'] = 'マツモトキヨシ'
invoice_df.loc[invoice_df['company_name'].str.contains('トモズ'), 'franchise_name'] = 'トモズ'
invoice_df.loc[invoice_df['company_name'].str.contains('イトーヨーカ堂'), 'franchise_name'] = 'イトーヨーカドー'
invoice_df.head(1)

,file_name,store_name,invoice_number,invoice_date,product_1,quantity_1,product_2,quantity_2,product_3,quantity_3,company_name,franchise_name
0,202009000096221.xlsx,トモズ お花茶屋店,202009000096221,44091,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ,トモズ


In [ ]:
# Create a function to extract store_name
def strip_store_name(string):
    # Convert set to string
    if isinstance(string, set):
        string = list(string)[0]

    # Convert value to string
    string = str(string)

    # Remove specific strings
    string = string.replace('マツモトキヨシ', '')
    string = string.replace('トモズ', '')
    string = string.replace('イトーヨーカ堂', '')

    # Remove spacing
    string = ''.join(string.split())

    return string

In [ ]:
invoice_df['store_name'] = invoice_df['store_name'].apply(strip_store_name)
invoice_df.head()

,file_name,store_name,invoice_number,invoice_date,product_1,quantity_1,product_2,quantity_2,product_3,quantity_3,company_name,franchise_name
0,202009000096221.xlsx,お花茶屋店,202009000096221,44091,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ,トモズ
1,202006000103601.xlsx,小田急マルシェ本厚木店,202006000103601,43997,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ,トモズ
2,202212000012341.xlsx,せんげん台東店,202212000012341,2022-12-09,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,マツモトキヨシ,マツモトキヨシ
3,202209000096221.xlsx,お花茶屋店,202209000096221,2022-09-05,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ,トモズ
4,202006000096221.xlsx,お花茶屋店,202006000096221,43991,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ,トモズ


### Add Address Column, Prefecture Column, & Municipality Column

In [ ]:
# Load the store data file
store_data = pd.read_csv('/content/drive/MyDrive/EY Recruitment Documents/new_shared_files/store_master.csv')
store_data.head()

,店舗ID,チェーン名,会社名,店舗名,住所
0,s001315,マツモトキヨシ,マツモトキヨシ,成増店,東京都板橋区成増２－１８－８奥山ビル１Ｆ
1,s010974,マツモトキヨシ,マツモトキヨシ,羽田エアポートガーデン店,東京都大田区羽田空港２－７－１羽田エアポートガーデン２Ｆ
2,s010395,マツモトキヨシ,マツモトキヨシ,二ツ木店,千葉県松戸市八ケ崎８－３４－３
3,s009453,マツモトキヨシ,マツモトキヨシ,池袋東口アネックス店,東京都豊島区南池袋１－２７－７
4,s007870,マツモトキヨシ,マツモトキヨシ,ららぽーと新三郷店,埼玉県三郷市新三郷ららシティ３－１－１ららぽーと新三郷１Ｆ


In [ ]:
# Check if there is any duplicated entry of a store location
store_data[store_data.duplicated(['会社名', '住所'])]

# Drop the duplicated row
store_data = store_data.drop_duplicates(subset = ['会社名', '住所'], keep = 'last').reset_index(drop = True)
#store_data[store_data.duplicated(['会社名', '住所'])]

In [ ]:
# Create a new column representing the combination of 'ID' and 'Category' in df1
invoice_df['company_store'] = invoice_df['company_name'] + '_' + invoice_df['store_name']
invoice_df.head()

# Create a new column representing the combination of 'Code' and 'Value' in df2
store_data['company_store'] = store_data['会社名'].astype(str) + '_' + store_data['店舗名']
store_data.head()

# Merge df2['Value'] into df1 based on matching 'Combination'
invoice_df_final = invoice_df.merge(store_data[['company_store', '住所']], on='company_store', how='left')

# Rename the address column to English
invoice_df_final = invoice_df_final.rename(columns={'住所': 'address'})
invoice_df_final['address'] = invoice_df_final['address'].astype(str)

invoice_df_final.head()
#invoice_df_final.info()

,file_name,store_name,invoice_number,invoice_date,product_1,quantity_1,product_2,quantity_2,product_3,quantity_3,company_name,franchise_name,company_store,address
0,202009000096221.xlsx,お花茶屋店,202009000096221,44091,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ,トモズ,トモズ_お花茶屋店,東京都葛飾区宝町２－３４－１６
1,202006000103601.xlsx,小田急マルシェ本厚木店,202006000103601,43997,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ,トモズ,トモズ_小田急マルシェ本厚木店,神奈川県厚木市中町１－１－１
2,202212000012341.xlsx,せんげん台東店,202212000012341,2022-12-09,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,マツモトキヨシ,マツモトキヨシ,マツモトキヨシ_せんげん台東店,埼玉県越谷市平方南町１９－１
3,202209000096221.xlsx,お花茶屋店,202209000096221,2022-09-05,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ,トモズ,トモズ_お花茶屋店,東京都葛飾区宝町２－３４－１６
4,202006000096221.xlsx,お花茶屋店,202006000096221,43991,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ,トモズ,トモズ_お花茶屋店,東京都葛飾区宝町２－３４－１６


In [ ]:
invoice_df_final.to_excel('/content/drive/MyDrive/EY Recruitment Documents/invoice_data_merged.xlsx', index=False)
invoice_df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4406 entries, 0 to 4405
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   file_name       4406 non-null   object 
 1   store_name      4406 non-null   object 
 2   invoice_number  4406 non-null   int64  
 3   invoice_date    4406 non-null   object 
 4   product_1       4406 non-null   object 
 5   quantity_1      4406 non-null   int64  
 6   product_2       4406 non-null   object 
 7   quantity_2      4406 non-null   int64  
 8   product_3       970 non-null    object 
 9   quantity_3      970 non-null    float64
 10  company_name    4406 non-null   object 
 11  franchise_name  4406 non-null   object 
 12  company_store   4406 non-null   object 
 13  address         4406 non-null   object 
dtypes: float64(1), int64(3), object(10)
memory usage: 516.3+ KB


## Create additional columns to use GeoPandas

In [ ]:
invoice_df_final = pd.read_excel('/content/drive/MyDrive/EY Recruitment Documents/invoice_data_merged.xlsx')
invoice_df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4406 entries, 0 to 4405
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   file_name       4406 non-null   object 
 1   store_name      4406 non-null   object 
 2   invoice_number  4406 non-null   int64  
 3   invoice_date    4406 non-null   object 
 4   product_1       4406 non-null   object 
 5   quantity_1      4406 non-null   int64  
 6   product_2       4406 non-null   object 
 7   quantity_2      4406 non-null   int64  
 8   product_3       970 non-null    object 
 9   quantity_3      970 non-null    float64
 10  company_name    4406 non-null   object 
 11  franchise_name  4406 non-null   object 
 12  company_store   4406 non-null   object 
 13  address         4392 non-null   object 
dtypes: float64(1), int64(3), object(10)
memory usage: 482.0+ KB




*   We identified that rows with company_store column == "マツモトキヨシ_ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店" had trouble finding corresponding address
  *   This was due to the a data corruption in the *store_master.csv* file shared from the data vendor
  * To remediate this issue, I have applied correction in the *store_master.csv* file
      * New: "ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店"
      * Old: "ｃｏｓ－ｍｅｄｉ　ｓｒａｔｉｏｎ西新宿メトロピア店". (= There was an unncesarry spaced between the letter "i" and "s")



In [ ]:
# Check if there are any issue in the dataset
invoice_df_final[invoice_df_final['address'].isnull()]

,file_name,store_name,invoice_number,invoice_date,product_1,quantity_1,product_2,quantity_2,product_3,quantity_3,company_name,franchise_name,company_store,address
4229,202104000050741.xlsx,ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,202104000050741,2021-04-16,メンズ・シャンプーM,100,レディース・シャンプーL,100,ラグジュアリ・シャンプーEx,100.0,マツモトキヨシ,マツモトキヨシ,マツモトキヨシ_ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,NaN
4230,202001000050741.xlsx,ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,202001000050741,43842,メンズ・シャンプーM,100,レディース・シャンプーL,100,ラグジュアリ・シャンプーEx,100.0,マツモトキヨシ,マツモトキヨシ,マツモトキヨシ_ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,NaN
4232,202007000050741.xlsx,ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,202007000050741,44036,メンズ・シャンプーM,100,レディース・シャンプーL,100,ラグジュアリ・シャンプーEx,100.0,マツモトキヨシ,マツモトキヨシ,マツモトキヨシ_ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,NaN
4234,202210000050741.xlsx,ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,202210000050741,2022-10-24,メンズ・シャンプーM,100,レディース・シャンプーL,100,ラグジュアリ・シャンプーEx,100.0,マツモトキヨシ,マツモトキヨシ,マツモトキヨシ_ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,NaN
4237,202301000050741.xlsx,ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,202301000050741,2023-01-05,メンズ・シャンプーM,100,レディース・シャンプーL,100,ラグジュアリ・シャンプーEx,100.0,マツモトキヨシ,マツモトキヨシ,マツモトキヨシ_ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,NaN
4238,202304000050741.xlsx,ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,202304000050741,2023-04-08,メンズ・シャンプーM,100,レディース・シャンプーL,100,ラグジュアリ・シャンプーEx,100.0,マツモトキヨシ,マツモトキヨシ,マツモトキヨシ_ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,NaN
4240,202010000050741.xlsx,ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,202010000050741,44128,メンズ・シャンプーM,100,レディース・シャンプーL,100,ラグジュアリ・シャンプーEx,100.0,マツモトキヨシ,マツモトキヨシ,マツモトキヨシ_ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,NaN
4242,202201000050741.xlsx,ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,202201000050741,2022-01-05,メンズ・シャンプーM,100,レディース・シャンプーL,100,ラグジュアリ・シャンプーEx,100.0,マツモトキヨシ,マツモトキヨシ,マツモトキヨシ_ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,NaN
4243,202207000050741.xlsx,ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,202207000050741,2022-07-21,メンズ・シャンプーM,100,レディース・シャンプーL,100,ラグジュアリ・シャンプーEx,100.0,マツモトキヨシ,マツモトキヨシ,マツモトキヨシ_ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,NaN
4244,202107000050741.xlsx,ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,202107000050741,2021-07-23,メンズ・シャンプーM,100,レディース・シャンプーL,100,ラグジュアリ・シャンプーEx,100.0,マツモトキヨシ,マツモトキヨシ,マツモトキヨシ_ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店,NaN


In [ ]:
# Fix the issue in the store data file and create a new file
store_data_mod = store_data.replace({'ｃｏｓ－ｍｅｄｉ　ｓｒａｔｉｏｎ西新宿メトロピア店': 'ｃｏｓ－ｍｅｄｉｓｒａｔｉｏｎ西新宿メトロピア店'})
store_data_mod['company_store'] = store_data_mod['会社名'].astype(str) + '_' + store_data_mod['店舗名']
store_data_mod.to_csv('/content/drive/MyDrive/EY Recruitment Documents/new_shared_files/store_master_mod.csv')
store_data_mod.head(1)

,店舗ID,チェーン名,会社名,店舗名,住所,company_store
0,s001315,マツモトキヨシ,マツモトキヨシ,成増店,東京都板橋区成増２－１８－８奥山ビル１Ｆ,マツモトキヨシ_成増店


In [ ]:
invoice_df_final = invoice_df.merge(store_data_mod[['company_store', '住所']], on='company_store', how='left')
invoice_df_final = invoice_df_final.rename(columns={'住所': 'address'})
invoice_df_final['address'] = invoice_df_final['address'].astype(str)
#invoice_df_final.head(1)
invoice_df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4406 entries, 0 to 4405
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   file_name       4406 non-null   object 
 1   store_name      4406 non-null   object 
 2   invoice_number  4406 non-null   int64  
 3   invoice_date    4406 non-null   object 
 4   product_1       4406 non-null   object 
 5   quantity_1      4406 non-null   int64  
 6   product_2       4406 non-null   object 
 7   quantity_2      4406 non-null   int64  
 8   product_3       970 non-null    object 
 9   quantity_3      970 non-null    float64
 10  company_name    4406 non-null   object 
 11  franchise_name  4406 non-null   object 
 12  company_store   4406 non-null   object 
 13  address         4406 non-null   object 
dtypes: float64(1), int64(3), object(10)
memory usage: 516.3+ KB


### Add municipal and prefecture columns

In [ ]:
# Define regular expressions for extracting prefecture and municipal strings
prefecture_regex = r'([一-龥]+都|[一-龥]+府|[一-龥]+県|[一-龥]+道)'
municipality_regex = r'([一-龥]+市|[一-龥]+区|[一-龥]+町|[一-龥]+村)'

# Function to extract prefecture string from address
def extract_prefecture(address):
    match = re.search(prefecture_regex, address)
    return match.group(0) if match else None

# Function to extract municipal string from address
def extract_municipality(address):
    match = re.search(municipality_regex, address)
    return match.group(0) if match else None

In [ ]:
# Apply the extraction functions to the 'Address' column and create new columns
invoice_df_final['prefecture'] = invoice_df_final['address'].astype(str).apply(extract_prefecture)
invoice_df_final['municipality'] = invoice_df_final['address'].astype(str).apply(extract_municipality)
invoice_df_final.head()

,file_name,store_name,invoice_number,invoice_date,product_1,quantity_1,product_2,quantity_2,product_3,quantity_3,company_name,franchise_name,company_store,address,prefecture,municipality
0,202009000096221.xlsx,お花茶屋店,202009000096221,44091,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ,トモズ,トモズ_お花茶屋店,東京都葛飾区宝町２－３４－１６,東京都,東京都葛飾区
1,202006000103601.xlsx,小田急マルシェ本厚木店,202006000103601,43997,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ,トモズ,トモズ_小田急マルシェ本厚木店,神奈川県厚木市中町１－１－１,神奈川県,神奈川県厚木市
2,202212000012341.xlsx,せんげん台東店,202212000012341,2022-12-09,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,マツモトキヨシ,マツモトキヨシ,マツモトキヨシ_せんげん台東店,埼玉県越谷市平方南町１９－１,埼玉県,埼玉県越谷市
3,202209000096221.xlsx,お花茶屋店,202209000096221,2022-09-05,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ,トモズ,トモズ_お花茶屋店,東京都葛飾区宝町２－３４－１６,東京都,東京都葛飾区
4,202006000096221.xlsx,お花茶屋店,202006000096221,43991,メンズ・シャンプーM,100,レディース・シャンプーL,100,NaN,NaN,トモズ,トモズ,トモズ_お花茶屋店,東京都葛飾区宝町２－３４－１６,東京都,東京都葛飾区


In [ ]:
invoice_df_final.info()
invoice_df_final.to_excel('/content/drive/MyDrive/EY Recruitment Documents/invoice_data_master.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4406 entries, 0 to 4405
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   file_name       4406 non-null   object 
 1   store_name      4406 non-null   object 
 2   invoice_number  4406 non-null   int64  
 3   invoice_date    4406 non-null   object 
 4   product_1       4406 non-null   object 
 5   quantity_1      4406 non-null   int64  
 6   product_2       4406 non-null   object 
 7   quantity_2      4406 non-null   int64  
 8   product_3       970 non-null    object 
 9   quantity_3      970 non-null    float64
 10  company_name    4406 non-null   object 
 11  franchise_name  4406 non-null   object 
 12  company_store   4406 non-null   object 
 13  address         4406 non-null   object 
 14  prefecture      4406 non-null   object 
 15  municipality    4406 non-null   object 
dtypes: float64(1), int64(3), object(12)
memory usage: 585.2+ KB
